### Define Loss Functions

In [677]:
"""Metrics to assess performance on ordinal classification task given class prediction
   using hyper plane loss techniques 
"""

# Authors: Bob Vanderheyden <rvanderh@us.ibm.com>
#          Ying Xie <yxie2@kennesaw.edu>
#         
# Contributor: Shayan Shamskolahi

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np

def hpall_mean_loss(y_true, y_pred, minlabel, maxlabel, margin=0.1, ordering_loss_weight=1):
    """ Evaluate the ordinal hyperplane ordering loss and point loss of the predictions y_pred\
        (using reduce mean).

        Parameters
        ----------
        y_true : array-like
        y_pred : array-like
        minlabel : integer
        maxlabel : integer
        margin : float
        ordering_loss_weight : float

        Returns
        -------
        loss: float
        A non-negative floating point value (best value is 0.0)
        
        Usage
        -------
        loss = hp_all_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
        print('Loss: ', loss.numpy()) # Loss: 0.7228571
        
        
        Usage with the `compile` API:
        
        ```python
        
        Example Keras wrapper for hp_all_loss:
        
        def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
            def ohpl(y_true, y_pred):
                return hpall_mean_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
            return ohpl

        loss = get_ohpl_wrapper(2,7,.3,1) # ordering_loss_weight must not be less that 1
        
        model = tf.keras.Model(inputs, outputs)
        model.compile(loss=hp_all_loss, optimizer='adam', loss=ohpl_point_loss)
        ```
        
    """
    
    min_label = tf.constant(minlabel, dtype=tf.float32)
    max_label = tf.constant(maxlabel, dtype=tf.float32)
    margin = tf.constant(margin, dtype=tf.float32) # centroid margin
    ordering_loss_weight = tf.constant(ordering_loss_weight, dtype=tf.float32) 
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    y_pred = tf.reshape(tf.transpose(y_pred),[-1,1])
    
    # OHPL ordering loss
    # one hot vector for y_true
    ords, idx = tf.unique(tf.reshape(y_true, [-1])) 
    num = tf.shape(ords)[0]
    y_true_1hot = tf.one_hot(idx, num)

    # mean distance for each class
    yO = tf.matmul(tf.transpose(y_pred),y_true_1hot)
    yc = tf.reduce_sum(y_true_1hot,0)
    class_mean = tf.divide(yO,yc)  

    # min. distance
    ords = tf.dtypes.cast(ords, tf.float32)
    ords0 = tf.reshape(ords, [-1,1])
    ords1 = tf.reshape(ords, [1,-1])
    
    min_distance = tf.subtract(ords0, ords1)
    # apply ReLU
    min_distance = tf.nn.relu (min_distance)
    
    # keeps min. distance
    keep = tf.minimum(min_distance,1)

    # distance to centroid     
    class_mean0 = tf.reshape(class_mean, [-1,1])
    class_mean1 = tf.reshape(class_mean, [1,-1])
    class_mean = tf.subtract(class_mean0, class_mean1)  
    # apply ReLU    
    class_mean = tf.nn.relu(class_mean)
    centroid_distance = tf.multiply(keep, class_mean)
    
    hp_ordering_loss = tf.subtract(min_distance,centroid_distance)
    # apply ReLU
    hp_ordering_loss = tf.nn.relu(hp_ordering_loss)
    hp_ordering_loss = tf.reduce_sum(hp_ordering_loss)
    
    # OHPL point loss
    # Centroid for point
    point_cent = tf.matmul(y_true_1hot, class_mean0)
    
    lower_bound = tf.subtract(min_label,y_true)
    lower_bound = tf.add(lower_bound,1)
    lower_bound = tf.multiply(lower_bound,1e9)
    # apply ReLU    
    lower_bound = tf.nn.relu(lower_bound)
    lower_bound = tf.add(margin, lower_bound)

    upper_bound = tf.subtract(y_true,max_label)
    upper_bound = tf.add(upper_bound,1)
    upper_bound = tf.multiply(upper_bound,1e9)
    # apply ReLU    
    upper_bound = tf.nn.relu(upper_bound)
    upper_bound = tf.add(margin, upper_bound)    

    upper_loss = tf.add(point_cent,upper_bound[:,tf.newaxis])
    upper_loss = tf.subtract(y_pred,upper_loss)
    # apply ReLU    
    upper_loss = tf.nn.relu(upper_loss)
    
    lower_loss = tf.add(lower_bound[:,tf.newaxis],y_pred)
    lower_loss = tf.subtract(point_cent,lower_loss)
    # apply ReLU    
    lower_loss = tf.nn.relu(lower_loss)
   
    hp_point_loss = tf.add(upper_loss, lower_loss)
    hp_point_loss = tf.reduce_mean(hp_point_loss)

    # aggregate ordering loss and point loss     
    mean_loss = tf.add(hp_point_loss,tf.multiply(ordering_loss_weight, hp_ordering_loss))
    
    return mean_loss

   
    """    
        References
        ----------
        .. [1] Vanderheyden, Bob and Ying Xie. Ordinal Hyperplane Loss. (2018). 
           2018 IEEE International Conference on Big Data (Big Data), 
           2018 IEEE International Conference On, 2337. https://doi-org.proxy.kennesaw.edu/10.1109/BigData.2018.8622079
    """

In [678]:
def hpall_sum_loss(y_true, y_pred, minlabel, maxlabel, margin=0.1, ordering_loss_weight=1):
    """ Evaluate the ordinal hyperplane ordering loss and point loss of the predictions y_pred\
        (using reduce sum).

        Parameters
        ----------
        y_true : array-like
        y_pred : array-like
        minlabel : integer
        maxlabel : integer
        margin : float
        ordering_loss_weight : float

        Returns
        -------
        loss: float
        A non-negative floating point value (best value is 0.0)
        
        Usage
        -------
        loss = hp_all_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,0.1)
        print('Loss: ', loss.numpy()) # Loss: 3.48
        
        
        Usage with the `compile` API:
        
        ```python
        
        Example Keras wrapper for hp_all_loss:
        
        def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
            def ohpl(y_true, y_pred):
                return hpall_sum_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
            return ohpl

        loss = get_ohpl_wrapper(0,4,1,1)
        
        model = tf.keras.Model(inputs, outputs)
        model.compile(loss=hp_all_loss, optimizer='adam', loss=ohpl_point_loss)
        ```
        
    """
    
    min_label = tf.constant(minlabel, dtype=tf.float32)
    max_label = tf.constant(maxlabel, dtype=tf.float32)
    margin = tf.constant(margin, dtype=tf.float32) # centroid margin
    ordering_loss_weight = tf.constant(ordering_loss_weight, dtype=tf.float32) 
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.dtypes.cast(y_true, y_pred.dtype)
    y_pred = tf.reshape(tf.transpose(y_pred),[-1,1])
    
    # OHPL ordering loss
    # one hot vector for y_true
    ords, idx = tf.unique(tf.reshape(y_true, [-1])) 
    num = tf.shape(ords)[0]
    y_true_1hot = tf.one_hot(idx, num)

    # mean distance for each class
    yO = tf.matmul(tf.transpose(y_pred),y_true_1hot)
    yc = tf.reduce_sum(y_true_1hot,0)
    class_mean = tf.divide(yO,yc)  

    # min. distance
    ords = tf.dtypes.cast(ords, tf.float32)
    ords0 = tf.reshape(ords, [-1,1])
    ords1 = tf.reshape(ords, [1,-1])
    
    min_distance = tf.subtract(ords0, ords1)
    # apply ReLU
    min_distance = tf.nn.relu (min_distance)
    
    # keeps min. distance
    keep = tf.minimum(min_distance,1)

    # distance to centroid     
    class_mean0 = tf.reshape(class_mean, [-1,1])
    class_mean1 = tf.reshape(class_mean, [1,-1])
    class_mean = tf.subtract(class_mean0, class_mean1)  
    # apply ReLU    
    class_mean = tf.nn.relu(class_mean)
    centroid_distance = tf.multiply(keep, class_mean)
    
    hp_ordering_loss = tf.subtract(min_distance,centroid_distance)
    # apply ReLU
    hp_ordering_loss = tf.nn.relu(hp_ordering_loss)
    hp_ordering_loss = tf.reduce_sum(hp_ordering_loss)
    
    # OHPL point loss
    # Centroid for point
    point_cent = tf.matmul(y_true_1hot, class_mean0)
    
    lower_bound = tf.subtract(min_label,y_true)
    lower_bound = tf.add(lower_bound,1)
    lower_bound = tf.multiply(lower_bound,1e9)
    # apply ReLU    
    lower_bound = tf.nn.relu(lower_bound)
    lower_bound = tf.add(margin, lower_bound)

    upper_bound = tf.subtract(y_true,max_label)
    upper_bound = tf.add(upper_bound,1)
    upper_bound = tf.multiply(upper_bound,1e9)
    # apply ReLU    
    upper_bound = tf.nn.relu(upper_bound)
    upper_bound = tf.add(margin, upper_bound)    

    upper_loss = tf.add(point_cent,upper_bound[:,tf.newaxis])
    upper_loss = tf.subtract(y_pred,upper_loss)
    # apply ReLU    
    upper_loss = tf.nn.relu(upper_loss)
    
    lower_loss = tf.add(lower_bound[:,tf.newaxis],y_pred)
    lower_loss = tf.subtract(point_cent,lower_loss)
    # apply ReLU    
    lower_loss = tf.nn.relu(lower_loss)
   
    hp_point_loss = tf.add(upper_loss, lower_loss)
    hp_point_loss = tf.reduce_sum(hp_point_loss)

    # aggregate ordering loss and point loss     
    sum_loss = tf.add(hp_point_loss,tf.multiply(ordering_loss_weight, hp_ordering_loss))
    
    return sum_loss


    """    
        References
        ----------
        .. [1] Vanderheyden, Bob and Ying Xie. Ordinal Hyperplane Loss. (2018). 
           2018 IEEE International Conference on Big Data (Big Data), 
           2018 IEEE International Conference On, 2337. https://doi-org.proxy.kennesaw.edu/10.1109/BigData.2018.8622079
    """

### Test the result:

In [679]:
loss = hpall_mean_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,1.0)
print('Loss: ', loss.numpy()) # Loss: 0.7228571

Loss:  3.2428572


In [680]:
loss = hpall_sum_loss([4,1,2,0,4,2,1], [6.0,3.1,5.2,1.0,4.0,2.2,3.7],0,4,.3,1.0)
print('Loss: ', loss.numpy()) # Loss: 3.48

Loss:  5.9


### Application in Keras (mean loss):

In [681]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [682]:
# prepare the data
pwd = !pwd
df = pd.read_csv('world_happiness_2015_2019.csv')
df.Score = df.Score.astype('int32')
df.drop(['Year'], axis=1, inplace=True)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781 entries, 0 to 781
Data columns (total 7 columns):
Score                           781 non-null int32
GDP per capita                  781 non-null float64
Social support                  781 non-null float64
Healthy life expectancy         781 non-null float64
Freedom to make life choices    781 non-null float64
Generosity                      781 non-null float64
Perceptions of corruption       781 non-null float64
dtypes: float64(6), int32(1)
memory usage: 45.8 KB


In [683]:
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

In [684]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([2, 3, 4, 5, 6, 7]), array([  5,  60, 135, 167, 108,  48], dtype=int64))
(array([2, 3, 4, 5, 6, 7]), array([ 2, 29, 67, 82, 54, 24], dtype=int64))


### Wrapper in action - Keras sequential model:

In [706]:
# example Keras wrapper for hpall_mean_loss

def get_ohpl_wrapper (min_label, max_label, margin, ordering_loss_weight):
    def ohpl(y_true, y_pred):
        return hpall_sum_loss(y_true, y_pred, min_label, max_label, margin, ordering_loss_weight)
    return ohpl

loss = get_ohpl_wrapper(2,7,.1,15) # ordering_loss_weight must not be less that 1

In [717]:
# Define and compile the model 
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(40, activation='relu', input_shape=(6, )))
model.add(Dropout(0.3))
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(24, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(loss=loss, optimizer=Adam(lr=0.005, decay=1e-4))
model.fit(X_train, y_train, epochs=2000, batch_size=12, shuffle=True)

Train on 523 samples
Epoch 1/2000
523/523 [==============================] - 1s 1ms/sample - loss: 192.6923
Epoch 2/2000
523/523 [==============================] - 0s 192us/sample - loss: 145.3539
Epoch 3/2000
523/523 [==============================] - 0s 194us/sample - loss: 124.1604
Epoch 4/2000
523/523 [==============================] - 0s 191us/sample - loss: 122.8710
Epoch 5/2000
523/523 [==============================] - 0s 191us/sample - loss: 119.9420
Epoch 6/2000
523/523 [==============================] - 0s 190us/sample - loss: 105.9725
Epoch 7/2000
523/523 [==============================] - 0s 187us/sample - loss: 100.7675
Epoch 8/2000
523/523 [==============================] - 0s 187us/sample - loss: 115.8574
Epoch 9/2000
523/523 [==============================] - 0s 187us/sample - loss: 100.5445
Epoch 10/2000
523/523 [==============================] - 0s 183us/sample - loss: 99.2243
Epoch 11/2000
523/523 [==============================] - 0s 187us/sample - loss: 101.9129
E

523/523 [==============================] - 0s 189us/sample - loss: 90.7948
Epoch 93/2000
523/523 [==============================] - 0s 196us/sample - loss: 79.6231
Epoch 94/2000
523/523 [==============================] - 0s 179us/sample - loss: 83.7439
Epoch 95/2000
523/523 [==============================] - 0s 200us/sample - loss: 89.1536
Epoch 96/2000
523/523 [==============================] - 0s 195us/sample - loss: 81.7026
Epoch 97/2000
523/523 [==============================] - 0s 181us/sample - loss: 82.3921
Epoch 98/2000
523/523 [==============================] - 0s 185us/sample - loss: 74.3287
Epoch 99/2000
523/523 [==============================] - 0s 183us/sample - loss: 83.4227
Epoch 100/2000
523/523 [==============================] - 0s 185us/sample - loss: 81.2412
Epoch 101/2000
523/523 [==============================] - 0s 175us/sample - loss: 85.3602
Epoch 102/2000
523/523 [==============================] - 0s 181us/sample - loss: 80.1394
Epoch 103/2000
523/523 [========

523/523 [==============================] - 0s 183us/sample - loss: 73.9047
Epoch 184/2000
523/523 [==============================] - 0s 181us/sample - loss: 81.5642
Epoch 185/2000
523/523 [==============================] - 0s 179us/sample - loss: 73.8042
Epoch 186/2000
523/523 [==============================] - 0s 183us/sample - loss: 80.9460
Epoch 187/2000
523/523 [==============================] - 0s 184us/sample - loss: 80.3362
Epoch 188/2000
523/523 [==============================] - 0s 177us/sample - loss: 79.0214
Epoch 189/2000
523/523 [==============================] - 0s 181us/sample - loss: 76.6166
Epoch 190/2000
523/523 [==============================] - 0s 187us/sample - loss: 80.4411
Epoch 191/2000
523/523 [==============================] - 0s 193us/sample - loss: 74.5903
Epoch 192/2000
523/523 [==============================] - 0s 183us/sample - loss: 75.5340
Epoch 193/2000
523/523 [==============================] - 0s 179us/sample - loss: 80.3349
Epoch 194/2000
523/523 [=

523/523 [==============================] - 0s 185us/sample - loss: 77.1885
Epoch 275/2000
523/523 [==============================] - 0s 179us/sample - loss: 71.0691
Epoch 276/2000
523/523 [==============================] - 0s 185us/sample - loss: 71.6132
Epoch 277/2000
523/523 [==============================] - 0s 189us/sample - loss: 77.3459
Epoch 278/2000
523/523 [==============================] - 0s 189us/sample - loss: 72.0529
Epoch 279/2000
523/523 [==============================] - 0s 183us/sample - loss: 73.7167
Epoch 280/2000
523/523 [==============================] - 0s 186us/sample - loss: 74.3603
Epoch 281/2000
523/523 [==============================] - 0s 187us/sample - loss: 77.1189
Epoch 282/2000
523/523 [==============================] - 0s 184us/sample - loss: 68.5683
Epoch 283/2000
523/523 [==============================] - 0s 187us/sample - loss: 81.0346
Epoch 284/2000
523/523 [==============================] - 0s 183us/sample - loss: 75.4544
Epoch 285/2000
523/523 [=

523/523 [==============================] - 0s 193us/sample - loss: 76.0837
Epoch 366/2000
523/523 [==============================] - 0s 183us/sample - loss: 69.2498
Epoch 367/2000
523/523 [==============================] - 0s 187us/sample - loss: 69.6616
Epoch 368/2000
523/523 [==============================] - 0s 179us/sample - loss: 67.1204
Epoch 369/2000
523/523 [==============================] - 0s 177us/sample - loss: 74.0129
Epoch 370/2000
523/523 [==============================] - 0s 183us/sample - loss: 74.0244
Epoch 371/2000
523/523 [==============================] - 0s 183us/sample - loss: 75.2462
Epoch 372/2000
523/523 [==============================] - 0s 181us/sample - loss: 68.8532
Epoch 373/2000
523/523 [==============================] - 0s 182us/sample - loss: 68.9441
Epoch 374/2000
523/523 [==============================] - 0s 181us/sample - loss: 71.3160
Epoch 375/2000
523/523 [==============================] - 0s 187us/sample - loss: 63.1371
Epoch 376/2000
523/523 [=

...

523/523 [==============================] - 0s 180us/sample - loss: 60.7204
Epoch 1633/2000
523/523 [==============================] - 0s 183us/sample - loss: 56.4182
Epoch 1634/2000
523/523 [==============================] - 0s 189us/sample - loss: 59.5443
Epoch 1635/2000
523/523 [==============================] - 0s 185us/sample - loss: 58.8495
Epoch 1636/2000
523/523 [==============================] - 0s 177us/sample - loss: 65.7730
Epoch 1637/2000
523/523 [==============================] - 0s 177us/sample - loss: 61.8386
Epoch 1638/2000
523/523 [==============================] - 0s 185us/sample - loss: 68.3666
Epoch 1639/2000
523/523 [==============================] - 0s 185us/sample - loss: 61.8091
Epoch 1640/2000
523/523 [==============================] - 0s 181us/sample - loss: 52.5424
Epoch 1641/2000
523/523 [==============================] - 0s 179us/sample - loss: 62.2428
Epoch 1642/2000
523/523 [==============================] - 0s 183us/sample - loss: 58.2184
Epoch 1643/2000

523/523 [==============================] - 0s 183us/sample - loss: 70.1928
Epoch 1723/2000
523/523 [==============================] - 0s 195us/sample - loss: 58.4027
Epoch 1724/2000
523/523 [==============================] - 0s 185us/sample - loss: 59.2937
Epoch 1725/2000
523/523 [==============================] - 0s 187us/sample - loss: 60.2947
Epoch 1726/2000
523/523 [==============================] - 0s 179us/sample - loss: 64.1341
Epoch 1727/2000
523/523 [==============================] - 0s 185us/sample - loss: 58.5006
Epoch 1728/2000
523/523 [==============================] - 0s 185us/sample - loss: 56.9059
Epoch 1729/2000
523/523 [==============================] - 0s 185us/sample - loss: 54.9957
Epoch 1730/2000
523/523 [==============================] - 0s 185us/sample - loss: 61.2201
Epoch 1731/2000
523/523 [==============================] - 0s 184us/sample - loss: 61.7986
Epoch 1732/2000
523/523 [==============================] - 0s 181us/sample - loss: 57.9952
Epoch 1733/2000

523/523 [==============================] - 0s 183us/sample - loss: 62.6307
Epoch 1813/2000
523/523 [==============================] - 0s 181us/sample - loss: 60.7297
Epoch 1814/2000
523/523 [==============================] - 0s 187us/sample - loss: 59.8840
Epoch 1815/2000
523/523 [==============================] - 0s 181us/sample - loss: 65.1955
Epoch 1816/2000
523/523 [==============================] - 0s 181us/sample - loss: 60.3863
Epoch 1817/2000
523/523 [==============================] - 0s 179us/sample - loss: 60.4901
Epoch 1818/2000
523/523 [==============================] - 0s 185us/sample - loss: 60.1613
Epoch 1819/2000
523/523 [==============================] - 0s 181us/sample - loss: 64.4396
Epoch 1820/2000
523/523 [==============================] - 0s 177us/sample - loss: 61.8557
Epoch 1821/2000
523/523 [==============================] - 0s 175us/sample - loss: 65.1459
Epoch 1822/2000
523/523 [==============================] - 0s 185us/sample - loss: 62.0945
Epoch 1823/2000

523/523 [==============================] - 0s 185us/sample - loss: 60.5181
Epoch 1903/2000
523/523 [==============================] - 0s 187us/sample - loss: 63.4504
Epoch 1904/2000
523/523 [==============================] - 0s 179us/sample - loss: 58.6707
Epoch 1905/2000
523/523 [==============================] - 0s 179us/sample - loss: 63.8557
Epoch 1906/2000
523/523 [==============================] - 0s 183us/sample - loss: 64.4356
Epoch 1907/2000
523/523 [==============================] - 0s 181us/sample - loss: 63.4495
Epoch 1908/2000
523/523 [==============================] - 0s 186us/sample - loss: 58.8229
Epoch 1909/2000
523/523 [==============================] - 0s 180us/sample - loss: 55.6529
Epoch 1910/2000
523/523 [==============================] - 0s 179us/sample - loss: 60.0311
Epoch 1911/2000
523/523 [==============================] - ETA: 0s - loss: 56.41 - 0s 186us/sample - loss: 60.2541
Epoch 1912/2000
523/523 [==============================] - 0s 185us/sample - loss:

Epoch 1992/2000
523/523 [==============================] - 0s 181us/sample - loss: 59.0593
Epoch 1993/2000
523/523 [==============================] - 0s 181us/sample - loss: 56.7507
Epoch 1994/2000
523/523 [==============================] - 0s 190us/sample - loss: 56.4528
Epoch 1995/2000
523/523 [==============================] - 0s 189us/sample - loss: 66.2264
Epoch 1996/2000
523/523 [==============================] - 0s 181us/sample - loss: 57.9584
Epoch 1997/2000
523/523 [==============================] - 0s 183us/sample - loss: 62.6157
Epoch 1998/2000
523/523 [==============================] - 0s 181us/sample - loss: 64.0477
Epoch 1999/2000
523/523 [==============================] - 0s 182us/sample - loss: 60.2150
Epoch 2000/2000
523/523 [==============================] - 0s 178us/sample - loss: 63.1921


#### Define the minimum class
min_class = min(np.unique(y_train))
y_train = np.array(y_train)

In [709]:
# Create matrix from on hot encoded training labels to use to calculate class centroids
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
onehot = onehot_encoder.fit_transform(y_train.reshape((-1, 1)))
onehot_inverse = 1/np.sum((onehot.T), axis=1)
new_y_train = onehot.T*onehot_inverse.reshape(-1,1)

In [710]:
# Score the training set
train_out = model.predict(X_train)
 

In [711]:
# Multiply centroid calculation matrix, new_y_train, by training set scores
train_cent = np.matmul(new_y_train, train_out)
train_cent

array([[-9.3272048 ],
       [-1.99663554],
       [-1.88011282],
       [-1.74404343],
       [-1.67826767],
       [-1.60158254]])

In [712]:
from sklearn.metrics import confusion_matrix 

rcenter = train_cent.T # create row matrix of centroids
pred = np.argmin(abs(train_out - rcenter), axis=1) + min_class

MAE = np.mean(abs(pred - y_train))
MZE = np.mean(abs(pred - y_train) > 0)   
print(MAE, MZE)

confusion_matrix(y_train, pred) 

0.39579349904397704 0.3824091778202677


array([[  5,   0,   0,   0,   0,   0],
       [  0,  57,   3,   0,   0,   0],
       [  0,  54,  40,  40,   1,   0],
       [  0,   3,   8, 126,  28,   2],
       [  0,   0,   0,  28,  57,  23],
       [  0,   0,   0,   1,   9,  38]], dtype=int64)

In [713]:
# Calculate new data model score
new_pred = model.predict(X_test)

In [714]:
# Identify the closest centroid
rcenter = train_cent.T # create row matrix of centroids
y_pred = np.argmin(abs(new_pred - rcenter), axis=1) + min_class

In [715]:
# calculate the mean absolute error and mean zero one error
mae = np.mean(abs(y_pred - y_test))
mze = np.mean(abs(y_pred - y_test) > 0)   
print(mae, mze)

0.5038759689922481 0.4844961240310077


In [716]:
# Confusion matrix
confusion_matrix(y_test, y_pred) 

array([[ 1,  1,  0,  0,  0,  0],
       [ 2, 21,  5,  1,  0,  0],
       [ 0, 19, 17, 31,  0,  0],
       [ 0,  2,  8, 53, 18,  1],
       [ 0,  0,  0, 20, 23, 11],
       [ 0,  0,  0,  1,  5, 18]], dtype=int64)